## Armado del dataset a nivel periodo, producto y cliente

In [2]:
from src.utils.utils import get_base_dir
import polars as pl

In [3]:
base_dir = get_base_dir()
base_dir

WindowsPath('C:/Users/lauta/Desktop/Lautaro/maestria_ds/labo3/repo-entrega')

## Importar base

In [4]:
# Paths
DATA_PATH_SELL_IN = base_dir/ "data/raw/sell-in.txt"
sell_in = pl.read_csv(DATA_PATH_SELL_IN, separator="\t")

## Agrupo por producto y cliente (por periodo también)

In [5]:
df = (
    sell_in
    .group_by(["periodo", "customer_id", "product_id"])
    .agg(
        pl.col("tn").sum().alias("tn")
    )
)
del sell_in

In [6]:
df

periodo,customer_id,product_id,tn
i64,i64,i64,f64
201811,10016,20412,0.01373
201910,10095,20043,0.03494
201911,10244,20220,0.01906
201911,10035,20100,0.41933
201709,10167,20161,0.10571
…,…,…,…
201806,10041,20003,0.62244
201903,10092,20069,0.27955
201907,10128,20078,0.21157


In [10]:
def completar_con_ceros_desde_primer_periodo_mejorado(df):
    """
    Versión mejorada que completa con ceros desde el primer período 
    en que aparece cada combinación (customer_id, product_id), pero también
    incluye productos que el socio nunca compró desde el primer período
    en que ese producto apareció en el sistema.
    """
    # Obtener el rango total de periodos
    periodos = df.select(pl.col("periodo")).unique().sort("periodo").to_series()
    
    # Obtener primer periodo en el que apareció cada producto en el sistema
    primer_periodo_por_producto_sistema = (
        df.group_by(["product_id"])
        .agg(pl.col("periodo").min().alias("primer_periodo_producto"))
    )
    
    # Obtener primer periodo en el que cada customer compró cada producto
    # (esto puede no existir para algunas combinaciones)
    primer_periodo_customer_producto = (
        df.group_by(["customer_id", "product_id"])
        .agg(pl.col("periodo").min().alias("primer_periodo_customer"))
    )
    
    # Obtener todos los customers y productos únicos
    customers = df.select(pl.col("customer_id")).unique()
    productos = df.select(pl.col("product_id")).unique()
    
    # Crear todas las combinaciones customer-producto posibles
    todas_combinaciones_customer_producto = customers.join(productos, how="cross")
    
    # Unir con la información de primer período del producto en el sistema
    combinaciones_con_primer_periodo = (
        todas_combinaciones_customer_producto
        .join(primer_periodo_por_producto_sistema, on="product_id", how="left")
        .join(primer_periodo_customer_producto, on=["customer_id", "product_id"], how="left")
    )
    
    # Para cada combinación, usar el primer período del customer si existe,
    # sino usar el primer período del producto en el sistema
    combinaciones_con_periodo_inicio = (
        combinaciones_con_primer_periodo
        .with_columns(
            pl.when(pl.col("primer_periodo_customer").is_not_null())
            .then(pl.col("primer_periodo_customer"))
            .otherwise(pl.col("primer_periodo_producto"))
            .alias("periodo_inicio")
        )
        .select(["customer_id", "product_id", "periodo_inicio"])
    )
    
    # Crear el dataframe con todos los períodos
    todos_los_periodos = pl.DataFrame({"periodo": periodos})
    
    # Generar todas las combinaciones desde el período de inicio correspondiente
    combinaciones = (
        combinaciones_con_periodo_inicio
        .join(todos_los_periodos, how="cross")
        .filter(pl.col("periodo") >= pl.col("periodo_inicio"))
        .select(["periodo", "customer_id", "product_id"])
    )
    
    # Unir contra el original para traer los valores de tn
    df_resultado = (
        combinaciones
        .join(df, on=["periodo", "customer_id", "product_id"], how="left")
        .fill_null(0)  # Rellenar con cero donde tn sea nulo
        .sort(["customer_id", "product_id", "periodo"])
    )
    
    return df_resultado

In [11]:
# Completamos df (desde primer período del producto)
df_mejorado = completar_con_ceros_desde_primer_periodo_mejorado(df)
print(f"   Combinaciones completas : {df_mejorado.shape[0]:,}")

   Combinaciones completas : 19,877,532


In [12]:
df_final = df_mejorado.clone()
print(f"Dataset final: {df_final.shape[0]:,} filas")

# Verificar que no hay valores nulos
print(f"Valores nulos en tn: {df_final.filter(pl.col('tn').is_null()).shape[0]}")
print(f"Valores cero en tn: {df_final.filter(pl.col('tn') == 0).shape[0]:,}")
print(f"Valores positivos en tn: {df_final.filter(pl.col('tn') > 0).shape[0]:,}")

Dataset final: 19,877,532 filas
Valores nulos en tn: 0
Valores cero en tn: 16,931,714
Valores positivos en tn: 2,945,818


In [13]:
# Guardar el resultado final
df_final.write_ipc((base_dir / "data/interim/sell_in_completo.feather"))